# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
## Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
## Check current working directory
print(os.getcwd())

## Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

## Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
## Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    ## print(file_path_list)


/home/workspace


#### Processing the files to create the new data file csv that will be used for Apache Casssandra tables

In [4]:
## Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
## For every filepath in the file path list 
for f in file_path_list:

## Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
## Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

## Create event_datafile_new csv that will be used to insert data into the\
## Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
## Check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

#### The event_datafile_new.csv, located within the Workspace directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
## Make a connection to a Cassandra instance in the local machine 
## (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

## Establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS annas
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('annas')
except Exception as e:
    print(e)


#### Three queries of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182   
#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [9]:
## Reset and Create tables
query = "drop table session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


## Create table session_library to meet query 1. for this table, column "sessionID" was used as a partition key because the queries will filter by this column.\   
## column "itemInSession" was used as clustering column to help make up a unique key.

query = "CREATE TABLE IF NOT EXISTS session_library "
query = query + "(artist_name text, song_title text, song_length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Create table artist_library to meet query 2. for this table, column "userId", "sessionId" were used as a COMPOSITE PRIMARY KEY(partition key),\ 
## because the queries will filter by these 2 columns to get the queries much faster.\   
## column "itemInSession" was used as clustering column to help sort out the data.    

query = "CREATE TABLE IF NOT EXISTS artist_library "
query = query + "(userId int, sessionId int, artist_name text, song_title text, itemInSession int, user_firstname text, user_lastname text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
## Create table user_library to meet query 3. for this table, column "song_title" was used as a PRIMARY KEY(partition key),\ 
## because the queries will filter by this column.\   
## column "userId" was used as clustering column to help make up a unique key.      
    
query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(userId int, song_title text, user_firstname text, user_lastname text, PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                 

In [10]:
## Use the new CSV file to insert values for every table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO session_library(artist_name, song_title, song_length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        
        query = "INSERT INTO artist_library(userId, sessionId, artist_name, song_title, itemInSession, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
        
        query = "INSERT INTO user_library(song_title, userId, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Meet the 3 queries

In [12]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "select artist_name, song_title, song_length from session_library WHERE sessionId = 139 AND itemInSession = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Tamba Trio Quem Quiser Encontrar O Amor 177.18812561035156


In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "select artist_name, song_title, iteminsession, user_firstname, user_lastname from artist_library WHERE userId = 10 AND sessionId = 182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.iteminsession, row.user_firstname, row.user_lastname)

                    

Down To The Bone Keep On Keepin' On 0 Sylvie Cruz
Three Drives Greece 2000 1 Sylvie Cruz
Sebastien Tellier Kilometer 2 Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3 Sylvie Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select user_firstname, user_lastname from user_library WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_firstname, row.user_lastname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Drop the tables before closing out the sessions

In [15]:
query = "drop table session_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table user_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Close the session and cluster connection

In [16]:
session.shutdown()
cluster.shutdown()